## Question 1: **Answers**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        # Creating a DataFrame
        data = pd.read_csv(input_data)
        engine = sqlite3.connect('sneaker_business_records.db')
        data.to_sql('sneakers', con=engine, index = False)
    except Exception as e:
        print(f"An error occurred: {e}")

data = "/content/sneaker_business.csv"
add_data_to_database(data)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
#  Connecting to the database
%sql sqlite:///sneaker_business_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM sneakers LIMIT 5;

 * sqlite:///sneaker_business_records.db
Done.


sneaker_id,model,price,release_date,colorway,size
1,Air Jordan 1 Retro High OG,160.0,2022-09-17,Black/White,10.5
2,Nike Air Force 1 '07 LV8,100.0,2022-09-17,White/Black,8.5
3,Adidas Yeezy Boost 350 V2,220.0,2022-09-18,Cream White,10.0
4,New Balance 990v5,160.0,2022-09-18,Grey/Black/White,11.5
5,Converse Chuck Taylor All-Star,70.0,2022-09-19,White/Black,9.5


## Question 2: **Answer**

In [7]:
%%sql
SELECT
    model,
    colorway
FROM sneakers
WHERE model LIKE '%Tempo';

 * sqlite:///sneaker_business_records.db
Done.


model,colorway
Brooks Hyperion Tempo,Black/White


## Question 3: **Answer**

In [8]:
%%sql
SELECT
    september_sneakers_value,
    october_sneakers_value
FROM (
    SELECT
        SUM(CASE WHEN STRFTIME('%m', release_date) = '09'
            THEN price ELSE 0 END) AS september_sneakers_value,

        SUM(CASE WHEN STRFTIME('%m', release_date) = '10'
            THEN price ELSE 0 END) AS october_sneakers_value
    FROM sneakers
) AS sneaker_value_calculation;

 * sqlite:///sneaker_business_records.db
Done.


september_sneakers_value,october_sneakers_value
1815.0,1665.0


## Question 4: **Answer**

In [9]:
%%sql
WITH nike_and_jordan AS (
    SELECT
        model,
        AVG(price) AS average_price
    FROM sneakers
    WHERE (price > 50 AND release_date > '2022-09-01')
           AND (model LIKE 'Nike%' OR  model LIKE '%jordan%')
   GROUP BY model
)
SELECT
    model,
    average_price,
    CASE
        WHEN average_price > 100 THEN 'Premium' ELSE 'Standard'
    END category
FROM nike_and_jordan;

 * sqlite:///sneaker_business_records.db
Done.


model,average_price,category
Air Jordan 1 Retro High OG,160.0,Premium
Nike Air Force 1 '07 LV8,100.0,Standard
Nike Air Zoom Pegasus 39,125.0,Premium
Nike Dunk Low,110.0,Premium


## Question 5: **Answer**

In [10]:
%%sql
WITH black_white AS (
    SELECT
        colorway,
        CAST(COUNT(model) AS REAL) AS count_black_white
    FROM sneakers
    WHERE colorway LIKE 'Black/White'
    GROUP BY colorway
),
navy_yellow AS (
    SELECT
        colorway,
        CAST(COUNT(model) AS REAL) AS count_navy_yellow
    FROM sneakers
    WHERE colorway LIKE 'Navy/Yellow'
    GROUP BY colorway
),
total_count AS (
    SELECT
        model,
        COUNT(*) AS total_sneakers
    FROM sneakers
)
SELECT
    bw.count_black_white,
    ny.count_navy_yellow,
    tc.total_sneakers,
    ROUND(((ny.count_navy_yellow + bw.count_black_white)/tc.total_sneakers * 100), 2)
    AS combined_percentage
FROM black_white bw
CROSS JOIN navy_yellow ny, total_count tc;

 * sqlite:///sneaker_business_records.db
Done.


count_black_white,count_navy_yellow,total_sneakers,combined_percentage
6.0,2.0,27,29.63


In [11]:
%%sql
SELECT
    (SELECT COUNT(model) FROM sneakers WHERE colorway LIKE 'Black/White')
    AS count_black_white,

    (SELECT COUNT(model) FROM sneakers WHERE colorway LIKE 'Navy/Yellow')
    AS count_navy_yellow,

    (SELECT COUNT(*) FROM sneakers) AS total_sneakers,

    ROUND(
        ((SELECT COUNT(model) FROM sneakers WHERE colorway LIKE 'Black/White') +
           (SELECT COUNT(model) FROM sneakers WHERE colorway LIKE 'Navy/Yellow')
        ) * 1.0 / (SELECT COUNT(*) FROM sneakers) * 100, 2
    ) AS combined_percentage;

 * sqlite:///sneaker_business_records.db
Done.


count_black_white,count_navy_yellow,total_sneakers,combined_percentage
6,2,27,29.63
